In [448]:
import numpy as np
import cv2

17. (5 баллов) Первая камера находится в начале координат, ее оптическая ось
    направлена вдоль оси z. Вторая камера сдвинута относительно первой на 1 в
    направлении оси x. Матрицы внутренних параметров обеих камер единичные.
    Сгенерировать случайный набор точек в трехмерном пространстве с z>0,
    спроектировать их на обе камеры (cv2.projectPoints), по проекциям найти
    необходимую матрицу (cv2.findEssentialMat).


In [449]:
num_points = 100
points_3d = np.random.rand(num_points, 3)

K = np.eye(3)


# T = np.array([[1.0, 0, 0, 1.0], [0, 1.0, 0, 0], [0, 0, 1.0, 0], [0, 0, 0, 1.0]])
T = np.array([-1.0, 0.0, 0.0])


points_img1, _ = cv2.projectPoints(
    points_3d, np.zeros(3), np.zeros(3), K, np.zeros(4))
points_img2, _ = cv2.projectPoints(points_3d, np.zeros(3), T, K, np.zeros(4))

E, mask = cv2.findEssentialMat(
    points_img1, points_img2, focal=1.0, pp=(0, 0), method=cv2.RANSAC
)

print("EssentialMat:")
print(E)

EssentialMat:
[[ 4.31987838e-15 -3.44196762e-15  5.86667355e-15]
 [ 1.10696884e-15 -4.82154168e-15 -7.07106781e-01]
 [ 2.39683575e-15  7.07106781e-01  2.42470593e-15]]


18. (5 баллов) По необходимой матрице все возможные позы первой камеры
    относительно второй, для этого использовать cv2.decomposeEssentialMat. Из
    получившихся 4 решений выбрать одно, используя условие, что трехмерные точки
    (сгенерированные в задаче 17) должны находиться в области видимости камер.


In [450]:
R1, R2, t = cv2.decomposeEssentialMat(E)
R1, R2, t

(array([[ 1.00000000e+00, -1.16863669e-14, -3.30218694e-15],
        [ 1.17155958e-14,  1.00000000e+00, -1.83186799e-15],
        [ 3.31937616e-15,  1.66533454e-15,  1.00000000e+00]]),
 array([[ 1.00000000e+00,  4.90709167e-15,  6.43316762e-15],
        [ 4.93632059e-15, -1.00000000e+00,  1.83186799e-15],
        [ 6.45035684e-15, -1.66533454e-15, -1.00000000e+00]]),
 array([[1.00000000e+00],
        [8.32595821e-15],
        [4.88486650e-15]]))

In [451]:
poses = [[R1, t], [R1, -t], [R2, t], [R2, -t]]
poses

[[array([[ 1.00000000e+00, -1.16863669e-14, -3.30218694e-15],
         [ 1.17155958e-14,  1.00000000e+00, -1.83186799e-15],
         [ 3.31937616e-15,  1.66533454e-15,  1.00000000e+00]]),
  array([[1.00000000e+00],
         [8.32595821e-15],
         [4.88486650e-15]])],
 [array([[ 1.00000000e+00, -1.16863669e-14, -3.30218694e-15],
         [ 1.17155958e-14,  1.00000000e+00, -1.83186799e-15],
         [ 3.31937616e-15,  1.66533454e-15,  1.00000000e+00]]),
  array([[-1.00000000e+00],
         [-8.32595821e-15],
         [-4.88486650e-15]])],
 [array([[ 1.00000000e+00,  4.90709167e-15,  6.43316762e-15],
         [ 4.93632059e-15, -1.00000000e+00,  1.83186799e-15],
         [ 6.45035684e-15, -1.66533454e-15, -1.00000000e+00]]),
  array([[1.00000000e+00],
         [8.32595821e-15],
         [4.88486650e-15]])],
 [array([[ 1.00000000e+00,  4.90709167e-15,  6.43316762e-15],
         [ 4.93632059e-15, -1.00000000e+00,  1.83186799e-15],
         [ 6.45035684e-15, -1.66533454e-15, -1.00000000e+

In [452]:
for i, pose in enumerate(poses):
    P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
    P2 = np.hstack((pose[0], pose[1]))
    points_triangulate = cv2.triangulatePoints(P1, P2, points_img1, points_img2)
    points_3d_reconstructed = points_triangulate[:3] / points_triangulate[3]
    if np.all(points_3d_reconstructed[2] > 0):
        print("Solution", i + 1, "is valid.")
        if i < 2:
            print("R1:", end=" ")
        else:
            print("R2:", end=" ")
        print(pose[0])
        print("t:", pose[1])

Solution 2 is valid.
R1: [[ 1.00000000e+00 -1.16863669e-14 -3.30218694e-15]
 [ 1.17155958e-14  1.00000000e+00 -1.83186799e-15]
 [ 3.31937616e-15  1.66533454e-15  1.00000000e+00]]
t: [[-1.00000000e+00]
 [-8.32595821e-15]
 [-4.88486650e-15]]


19. (5 баллов) Найти матрицу внутренних параметров камеры изображения
    https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing, предполагая, что оптическая ось проходит строго через центр изображения, а
    фокусное расстояние в эквиваленте 35мм равно 0.64 дюйма. Почему результат
    отличается от параметров, полученных методом калибрации камеры
    https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing?


In [ ]:
<?xml version="1.0"?>
<opencv_storage>
<calibration_time>"Sun Jan 20 18:22:54 2019"</calibration_time>
<image_width>960</image_width>
<image_height>540</image_height>
<board_width>7</board_width>
<board_height>9</board_height>
<square_size>1.</square_size>
<flags>16384</flags>
<camera_matrix type_id="opencv-matrix">
  <rows>3</rows>
  <cols>3</cols>
  <dt>d</dt>
  <data>
    4.2581151397691417e+02 0. 4.8175802555527360e+02 
    0. 4.3533255745518812e+02 2.6743704185634374e+02
    0. 0. 1.</data></camera_matrix>
<distortion_coefficients type_id="opencv-matrix">
  <rows>14</rows>
  <cols>1</cols>
  <dt>d</dt>
  <data>
    4.4429001834994422e+00 2.7649030467662392e+00
    -4.5203901289078106e-03 -1.8217805324766470e-03 0.
    4.7056576535214534e+00 3.9343291290497260e+00 3.3910745370687528e-01
    0. 0. 0. 0. 0. 0.</data></distortion_coefficients>
<avg_reprojection_error>4.9917025890850247e-01</avg_reprojection_error>
</opencv_storage>

In [456]:
from matplotlib import pyplot as plt

img = cv2.imread("images\\GOPR01170000.jpg")
f = 0.0254 * 0.64 * 1000
pix_x = 36 / img.shape[1]
pix_y = 24 / img.shape[0]
K = np.array(
    [[f / pix_x, 0, img.shape[1] / 2], [0, f / pix_y, img.shape[0] / 2], [0, 0, 1]]
)
K

array([[433.49333333,   0.        , 480.        ],
       [  0.        , 365.76      , 270.        ],
       [  0.        ,   0.        ,   1.        ]])